In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
model = "gpt-4o-mini"

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model,
    output_type=str
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=model,
    output_type=str
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model,
    output_type=str
)

In [7]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email.")

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Enhance Your SOC 2 Compliance with AI-Driven Solutions

Hi [Recipient's Name],

I hope this email finds you well. My name is [Your Name], and I represent ComplAI, a pioneering company dedicated to simplifying the SOC 2 compliance process.

In today's rapidly evolving digital landscape, ensuring your organization adheres to compliance standards is not just a necessity—it's a competitive advantage. Our AI-powered SaaS tool streamlines audit preparations and helps maintain ongoing compliance effortlessly.

Key benefits of our solution include:

- **Automated Documentation**: Save time and resources with our automated documentation generation.
- **Real-Time Monitoring**: Get insights and alerts to address potential compliance issues proactively.
- **Customizable Frameworks**: Tailor your compliance strategies to fit your unique business needs.

I would love the opportunity to discuss how ComplAI can assist [Recipient's Company] in effectively managing your SOC 2 compliance. Are yo

In [8]:
message = "Write a cold sales email."

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. I’m reaching out to introduce you to ComplAI, a powerful SaaS solution designed to simplify the SOC 2 compliance process and streamline your audit preparations.

In today’s regulatory landscape, ensuring compliance can be both time-consuming and complex. ComplAI leverages advanced AI technology to automate documentation, track compliance metrics, and generate real-time reports, enabling your team to focus on what truly matters—growing your business.

Here are a few key benefits of using ComplAI:

- **Automated Documentation:** Reduce manual effort with smart templates and automated workflows.
- **Real-Time Insights:** Track your compliance status effortlessly with our intuitive dashboard.
- **Audit Readiness:** Prepare for audits with confidence, knowing you have everything under control.

I would love to schedule a brief call to discuss how ComplAI can specifically cater

In [6]:
sales_picker = Agent(
    name="sales picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=model
)

In [10]:
message = "Write a cold sales email."

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best cold sales email:\n{best.final_output}")

Best cold sales email:
Subject: Unlock the Secret to SOC2 Compliance Without Losing Your Mind! 🕵️‍♂️

Hey [Recipient's Name],

Ever feel like SOC2 compliance is like trying to find a unicorn in a haystack? 🦄 You know it’s out there, but good luck wrangling it!

At ComplAI, we help businesses like yours turn the SOC2 compliance maze into a walk in the park. 🌳 With our powerful AI-driven tool, you'll breeze through audits with the elegance of a gazelle—no more sleepless nights worrying about compliance issues (goodbye, caffeine overload!).

Imagine having a compliance assistant that never takes a coffee break, remembers every regulation, and sends you reminders with just the right amount of sass. Sounds pretty great, right? Coffee not included, but we guarantee you'll need less of it! ☕💼

Let’s set up a quick call so you can see how ComplAI can elevate your compliance game and bring a smile back to your face. 

Looking forward to your thoughts!

Best,  
[Your Name]  
[Your Title]  
Compl

In [7]:
@function_tool
def send_email(body: str):
    print("send_email called with body:", body)
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kaya.atsy@gmail.com")
    to_email = To("kaya.atsy@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    print("Built mail object:", mail)
    response = sg.client.mail.send.post(request_body=mail)
    print("SendGrid response:", response.status_code, response.body)
    return "Email successfully sent"


In [8]:
send_email

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FF26ED0720>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [9]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email.")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FF26ED0EA0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [10]:
description = "Write a cold sales email."

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FF26ED1260>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001FF26ED16C0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', des

In [11]:
instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

In [12]:
sales_manager = Agent(name="Sales manager", instructions=instructions, tools=tools, model=model)

message = "Send a cold sales email addressed to 'Dear CEO'"

#with trace("Sales Manager"):
    #result = await Runner.run(sales_manager, message)

In [13]:
#Handoffs

In [14]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design.\
When converting to HTML, pay extra attention to any placeholder text inside the brackets.\
If such placeholder brackets exist in the body passed to you, simply change them to a more general word according to\
the info inside the brackets and the overall tone of the email body.\
For example, if you see [CEO's Name] in a body, you must simply change it to CEO without brackets."

In [15]:
subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=model)
html_converter_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

In [16]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("kaya.atsy@gmail.com")
    to_email = To("kaya.atsy@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return "Successfully sent email"

In [17]:
tools = [subject_tool, html_converter_tool, send_html_email]

In [18]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(name="Email Manager", instructions=instructions, model=model, tools=tools, handoff_description="Convert an email to HTML and send it")

In [19]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [20]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools a second time to draft new email bodies if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

In [21]:
sales_manager = Agent(name="Sales Manager", instructions=sales_manager_instructions, model=model, tools=tools, handoffs=handoffs)

message = "Send out a cold sales email addressed to 'Dear CEO' from Alice. Make sure email bodies do not contain placeholder text/brackets."

#with trace("Automated SDR"):
    #result = await Runner.run(sales_manager, message)

In [22]:
#Guardrails

In [30]:
from pydantic import BaseModel
from agents import input_guardrail, GuardrailFunctionOutput

In [25]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

In [28]:
guardrail_agent = Agent(
    name="Name Checker",
    model=model,
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput
)

In [31]:
@input_guardrail
async def guardail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message

    return GuardrailFunctionOutput(output_info={"found_name": result.final_output}, tripwire_triggered=is_name_in_message)

In [32]:
careful_sales_manager = Agent(
    name="Sales Manager",
    model=model,
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    input_guardrails=[guardail_against_name]
)

In [35]:
message = "Send out a cold sales email to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [36]:
message = "Send out a cold sales email to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)